In [2]:
#不运行参数定义的行,吧参数定义行和实体运行行分开，把args.改成args.就能运行例子而不是写py
args.region = '/public/home/lizw/task/pore_c/porec_result2/bed/all_chr.bed'
args.anchor = '/public/home/lizw/task/pore_c/porec_result2/bed/peri.all.bed'
args.align_merge_part = '/public/home/lizw/task/pore_c/porec_1000_filter_mainchr_result/porecplot/DpnII_run04.order_2_merge_part.csv'
args.keepall = '/public/home/lizw/task/pore_c/porec_1000_filter_mainchr_result/porecplot/DpnII_run04.order_2_sorted_orderall_fornormailize.csv'
args.clusterheat = True
args.binsize = 500000
args.calculate = 'raw'
args.prefix = 'test1'
args.mark = '/public/home/lizw/task/pore_c/pipeline_test_v1/setfiles/test_mark.csv'
args.bedgraphs = ['/public/home/lizw/data/IMPORTANT_genome/arabidopsis/modification/H3K4me2-DRX066771.bw.bg','/public/home/lizw/data/IMPORTANT_genome/arabidopsis/modification/H3K27me3-SRX648279.bw.bg']
args.bg_labels = ['H3K4me2','H3K27me3']
args.bg_ylims = [None,None]
args.plotkind = 'bar'
args.sites = []
args.site_labels = [None]
args.site_ylims = [None]
args.anchormode = 'and'
args.genemodel = ''
args.slice_cluster = True

In [1]:
%%writefile '/public/home/lizw/task/pore_c/tools/porecscripts/v2.15/porecplot_draw.py'

#@Author: Zhuowen Li
#@LastEdit: 2021/12/28 下午2:16:21
#@Version: v2.14
#@Description: change binnor calculation method

#@Author: Zhuowen Li
#@LastEdit: 2021/10/5 下午2:42:17
#@Version: v2.12
#@Description: change anchorcluster color

#@Author: Zhuowen Li
#@LastEdit: 2021/10/1 下午12:16:34
#@Version: v2.11
#@Description: generate matrix independently with heatmap draw,modify the plot specially for website

#@Author: Zhuowen Li
#@LastEdit: 2021/9/24 下午3:37:52
#@Version: v2.10
#@Description: change the anchorcluster color back to mupltiple one

#@Author: Zhuowen Li
#@LastEdit: 2021/9/10 下午1:05:04
#@Version: 
#@Description: change "fragment_mid" to "pos"

#@Author: Zhuowen Li
#@LastEdit: 2021/9/4 下午8:51:14
#@Version: v2.7
#@Description: anchorcluster two color 

#@Author: Zhuowen Li
#@LastEdit: 2021/8/31 下午11:04:38
#@Version: v2.6
#@Description: 
#1.mark file columns: chrom start end name color
#2.provide optional sorting for the heatmap: maxily keep all reads in the plot



#@Author: Zhuowen Licd
#@LastEdit: 2021/8/21 下午8:22:05
#@Version: v2.4
#@Description: 
#1.step sample according to the combinations of anchors, for cluster heatmap drawing


'''
Version description_v9: 
1.
output the read matrix after sorted by clustr heatmap

Basic Function:
-plot porec/hic interactions along with reads.
-anchor one site or more to find the site interact with it.(when anchor serval sites, can choose anchormode = and/or)
-compare different porec/hic interaction.
-normalize with bin infomation.
-Modification in bedgraph format(eg. H3K4me1), or special site distribution(eg.DpnII cutting site) listed in format ['chrom'\t'site'] can be draw along side with same binsize.

Attention:
Interaction plots (including line or bar plot in porecplot, detail and clusterheat) only count 0/1 (exist or not) for per read in per bin to mask the local interaction.
The heatmap would keep the interaction pattern within bins with different degree of color

'''

import argparse
#porec basic explore
parser = argparse.ArgumentParser(description='Exploring the interaction of special parts using poreC data. Using the anchor FILE as anchor filtering condition. The pipeline will choose the reads that contact all the anchors in the anchor FILE, which can tell you the level of direct interaction between those anchors.')

#porecplot basic infomation
parser.add_argument('--align_merge_part',type=str,help='align_merge_part input')
parser.add_argument('--keepall',type=str,help='for normalization')
parser.add_argument('--region', type=str, help='Input the explore region in bed format, should be written in order. Necessary for porecplot, detail and clusterheat option. The chr columns should be the same format as align file.' )
parser.add_argument('--anchor',type=str,help='The anchors you anchor to filter the reads. Necessary for poreplot and detail option')
parser.add_argument('--anchormode',type=str,help='anchor reads that catch sites with all anchor points[anchormode = and] or one of then [anchormode = or]')

#plots options
parser.add_argument('--draw',action = 'store_true',help='Draw or not')
parser.add_argument('--mark',type=str,help='Show mark position with color')
parser.add_argument('--plotkind',type=str,help='Draw line plot or bar plot of porecplot')
parser.add_argument('--genemodel',type=str,nargs='?', const='',help='Draw gene model in porecplot')
parser.add_argument('--clusterheat',action='store_true',help='Draw clusterheat or not')
parser.add_argument('--anchorcluster',action='store_true',help='Draw anchorcluster or not')

#calculation parameter
parser.add_argument('--anchor_stat',action = 'store_true',help = 'do anchor statistics')
parser.add_argument('--step_sample',type = float,help = 'step sample according to the combinations of anchors, for cluster heatmap drawing')
parser.add_argument('--calculate',type=str,help='Choose different calculation methods for interaction value, raw/logy/binnor/binnorlogy')
parser.add_argument('--binsize',type=int,help='Decide plotting bin size')
parser.add_argument('--ylim',help='set the ylim for distribution plot in porecplot')

#other bg_ylimss
parser.add_argument('--bedgraphs',nargs='*',help='bedgraph draw, seperated by space in order')
parser.add_argument('--bg_labels',nargs='*',help='bedgraph labels, seperated by space in order ')
parser.add_argument('--bg_ylims',nargs='*',help='bedgraph ylims, seperated by space in order')

#special site distribution
parser.add_argument('--sites',nargs='*',help='site draw, seperated by space in order')
parser.add_argument('--site_labels',nargs='*',help='site labels, seperated by space in order ')
parser.add_argument('--site_ylims',nargs='*',help='site ylims')

#output parameter
parser.add_argument('--prefix',type=str,help='output file prefix')
parser.add_argument('--format',type=str,help='porecplot format')
                   
args = parser.parse_args()

from porecplot_draw_func import *
import numpy as np
import math
import seaborn as sns


import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
font_dirs = ["/public/home/lizw/software/font"]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 20

from matplotlib.gridspec import GridSpec
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap, Normalize



region_df = region_reads(args.region)
anchor = pd.read_csv(args.anchor,sep="\t",header=None,usecols=[0,1,2,3],names=['chrom','start','end','name'],converters={'chrom':str, 'start':int, 'end':int,'name':str},comment="#")
df_merge_part = pd.read_parquet(args.align_merge_part)
anchor_reads_detail_df = anchor_detail(anchor,df_merge_part,args.prefix)

if args.anchor_stat == True:
    anchor_statics(anchor_reads_detail_df,args.prefix)

merge_keepall_part = pd.read_parquet(args.keepall)
merge_keepall_part.rename(columns={'pos':'site'},inplace=True)

anchor_interval_count,anchor_filter_merge,anchor_and_list = anchor_reads(anchor,anchor_reads_detail_df,df_merge_part,region_df,'pos',args.binsize,args.anchormode,args.prefix,True,step_sample=args.step_sample)
anchor_binnor_merge_all_sort = anchor_calcu(merge_keepall_part,anchor_interval_count,region_df,args.binsize,args.prefix,True)


if args.anchorcluster:
    viridis = cm.get_cmap('OrRd', 6)
    newcolors = viridis(np.linspace(0, 1, 256))
    white = np.array([255/255, 255/255, 255/255, 1])
    newcolors[:50, :] = white
    newcmp1 = ListedColormap(newcolors)

    anchor_and_detail = anchor_reads_detail_df.query('read_name in @anchor_and_list')
    read_anchor_matrix = anchor_and_detail.groupby(['read_name','anchor_name'])['pos'].count().unstack().fillna(0).reindex(anchor['name'].to_list(), axis=1)
    #make a only 0_1 matrix for sort, to make the same combinations together
    read_anchor_matrix_01 = read_anchor_matrix != 0
    read_anchor_matrix_01_sort = read_anchor_matrix_01.sort_values(by=anchor['name'].to_list())
    #make the read list of comb sort as ref,and sort the orginal matrix(contain the information of locus self interaction within one read)
    read_comb_sort_list = read_anchor_matrix_01_sort.reset_index().iloc[:,0].to_list()
    read_anchor_matrix_sort=read_anchor_matrix.reindex(read_comb_sort_list,axis=0)
    fig_anclu, ax_anclu = plt.subplots(1,figsize=(20,20))
    g_anclu = sns.heatmap(read_anchor_matrix_sort.reset_index().iloc[:,1:],vmin=0-0.5,vmax=5+0.5,ax=ax_anclu,cmap=newcmp1,cbar=False)
    #g_anclu.ax_heatmap.tick_params(tick2On=False, labelsize=False)
#     ax_anclu.set_xlabels('')
#     ax_anclu.set_yticklabels([])
    plt.savefig(args.prefix+'_anchorcluster.'+ args.format,format = args.format,dpi=300,bbox_inches = 'tight')

if args.draw == True:
    #define the absolute length of each part, and calculate the proportion, so the main plot would stay the same while other tracks added
    heat_concat,read_concat = heat_df(anchor_binnor_merge_all_sort,anchor_filter_merge)
    heat_concat.columns = list(range(0,len(heat_concat.columns)))
    bg_len = len(args.bedgraphs) if args.bedgraphs else 0
    site_len = len(args.sites) if args.sites else 0 
    yflen = 12+bg_len+site_len
    heat_concat_sort = heat_concat.sort_values(by=heat_concat.columns.to_list())
    viridis = cm.get_cmap('BuPu', 6)
    newcolors2 = viridis(np.linspace(0, 1, 256))
    white = np.array([255/255, 255/255, 255/255, 1])
    newcolors2[:50, :] = white
    newcmp2 = ListedColormap(newcolors2)

    if len(heat_concat_sort) >= 10:
        cluster_top_para = 4.2
        cluster_bottom_para = colorbar_bottom_para = 10.5
        colorbar_top_para = 5
        clustermap = sns.clustermap(heat_concat_sort,figsize=(30,yflen),cmap = newcmp2,col_cluster=False,row_cluster=args.clusterheat,vmin=0-0.5,vmax=5+0.5,dendrogram_ratio=0.1,cbar_pos=None)
        clustermap.ax_heatmap.tick_params(tick2On=False, labelsize=False)
        cluster_order = clustermap.dendrogram_row.reordered_ind
        #output the clustermatrix
        heat_concat_sort_cluster = heat_concat_sort.reindex(cluster_order)
        heat_concat_sort_cluster.to_csv(args.prefix+'_heat_matrix.csv')
        
        ax_cluster = clustermap.fig.gca()
        ax_cluster.set_xticks([])
        #clustermap.gs.update(top=1-cluster_top_para/yflen, bottom=1-cluster_bottom_para/yflen,left=-0.00000000000000000001,right = 0.9)
        clustermap.gs.update(top=1-cluster_top_para/yflen, bottom=1-cluster_bottom_para/yflen,left=0.0077,right = 0.9)

        #mainplot
        mainplot = clustermap.fig
        #colorbar
        gs1 = GridSpec(1,1,top=1-colorbar_top_para/yflen, bottom=1-colorbar_bottom_para/yflen,left = 0.95, right = 1) 
        ax_color = mainplot.add_subplot(gs1[0])
        ax_color.set_yticklabels([])
        remove_draw(ax_color,tick=True,spine=True,locator=True)
        cbnorm = Normalize(vmin=0-0.5,vmax=5+0.5,clip=False)
        cb = plt.colorbar(cm.ScalarMappable(norm=cbnorm, cmap=newcmp2),fraction=1,ax=ax_color,ticks=np.arange(6))

    else:
        cluster_top_para = cluster_bottom_para = colorbar_top_para = colorbar_bottom_para = 0
        with open (f'{args.prefix}.report','a+') as report:
            report.write('Reads count lower than 10! Remove cluster heatmap.')
        mainplot = plt.figure()


    gs2 = GridSpec(1,1, top=1,bottom=1-4.8/yflen,left = 0.10,right = 0.9)
    #create axes within this new gridspec
    ax_binsum = mainplot.add_subplot(gs2[0])
    porec_draw(anchor_binnor_merge_all_sort,ax_binsum,args.calculate,args.plotkind,args.ylim)

    below_cluster_para = cluster_bottom_para + 0.5
    #bedgraph information
    if args.bedgraphs:
        bg_top = 1-below_cluster_para/yflen
        bg_bottom = 1-(bg_len+below_cluster_para)/yflen
        gs3 = GridSpec(bg_len,1,top=bg_top,bottom=bg_bottom,left=0.10,right = 0.9,hspace=0.2)
        track_draw(args.bedgraphs,args.bg_labels,args.bg_ylims,mainplot,region_df,args.binsize,gs3,0,'bg') 


    if args.sites:
        site_top = 1-(bg_len+0.2+below_cluster_para)/yflen
        site_bottom = 1-(bg_len+0.2+site_len+below_cluster_para)/yflen
        gs4 = GridSpec(site_len,1,top=site_top,bottom=site_bottom,left=0.10,right = 0.9,hspace=0.2)
        track_draw(args.sites,args.site_labels,args.site_ylims,mainplot,region_df,args.binsize,gs4,bg_len,'site')   

    #anchor and mark information
    region_df_cum = region_df.copy()
    region_df_cum['cum'] = anchor_binnor_merge_all_sort.groupby('region_index').count()['index'].cumsum()
    region_figlensum = anchor_binnor_merge_all_sort.groupby('region_index').count()['index'].sum()
    gs5 = GridSpec(1,region_figlensum,top=1,bottom=0.0,left=0.10,right = 0.9)
    
    kw = dict()
    if args.genemodel:
        gene_isoform = pd.read_csv(args.genemodel, sep='\t', names=['chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts'],comment="#")
        kw['gene'] = gene_isoform
    if args.mark:
        mark = pd.read_csv(args.mark,sep="\t",header=None,names=['chrom','start','end','name','color'],converters={'chrom':str, 'start':int, 'end':int,'color':str},comment="#")
        kw['mark'] = mark

    rec_draw(region_df_cum,mainplot,anchor,args.binsize,gs5,yflen,args.anchormode,**kw)
    plt.savefig(args.prefix+'.'+ args.format,format = args.format,dpi=500,bbox_inches = 'tight')

with open (f'{args.prefix}.report','a+') as report:
    report.write('Succesfully finish!')


Writing /public/home/lizw/task/pore_c/tools/porecscripts/v2.15/porecplot_draw.py


In [19]:
list(range(0,5))

[0, 1, 2, 3, 4]

In [3]:
import pandas as pd

In [11]:
df_merge_part = pd.read_parquet('/public/home/lizw/task/pore_c/porec_1000_filter_mainchr_result/porecplot_v3/porec_pre_hic_v2.8/DpnII_hic.hic_merge_part.parquet')

In [12]:
df_merge_part

,index,read_name,chrom,pos
0,44279054,SRR1029605.1.10000000,3,1897711
1,44279054,SRR1029605.1.10000000,3,1905603
2,46458481,SRR1029605.1.100000002,3,6817354
3,46458481,SRR1029605.1.100000002,3,4556657
4,46557315,SRR1029605.1.100000005,3,9571905
...,...,...,...,...
185405831,37650836,SRR1029605.1.99999983,2,17851275
185405832,11377168,SRR1029605.1.99999992,1,17200095
185405833,11377168,SRR1029605.1.99999992,1,17200474
185405834,1971766,SRR1029605.1.99999994,1,2207396


In [2]:
import pandas as pd

In [3]:
df = pd.read_parquet('/public/home/lizw/task/pore_c/porec_1000_filter_mainchr_result/porecplot_v3/porec_pre_v2.9/DpnII_run08.order_2_merge_part.parquet')

In [4]:
df

,index,chrom,read_name,pos
0,1,3,2910745e-eb49-4901-a1f2-4491ede1d0d7,20076841
1,2,1,2910745e-eb49-4901-a1f2-4491ede1d0d7,18952877
2,3,3,2910745e-eb49-4901-a1f2-4491ede1d0d7,19596981
3,5,2,2910745e-eb49-4901-a1f2-4491ede1d0d7,17085746
4,6,3,2910745e-eb49-4901-a1f2-4491ede1d0d7,18879034
...,...,...,...,...
22810633,36378731,2,05d0ead4-62e1-40fe-94f7-4a3b7165598d,972242
22810634,36378732,2,05d0ead4-62e1-40fe-94f7-4a3b7165598d,4964832
22810635,36378734,2,05d0ead4-62e1-40fe-94f7-4a3b7165598d,4703718
22810636,36378737,2,4307c964-4e17-4ed0-9e4d-aca5ea93f43c,19407635
